<a href="https://colab.research.google.com/github/TNaingHtwe/GraSPP-25S-climatechange/blob/main/notebooks/20250614_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/TNaingHtwe/GraSPP-25S-climatechange.git
%cd GraSPP-25S-climatechange/notebooks

Cloning into 'GraSPP-25S-climatechange'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 370 (delta 49), reused 57 (delta 25), pack-reused 248 (from 1)
Receiving objects: 100% (370/370), 34.09 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (152/152), done.
/content/GraSPP-25S-climatechange/notebooks


In [5]:
import pandas as pd

In [15]:
# Load GHG emission data from one level up in data/processed
ghg_df = pd.read_csv("../data/processed/edgar_ghg_emission_processed.csv")
ghg_df.head()

,Unnamed: 0,Country,Year,Emission (MtCO2eq/yr)
0,0,GLOBAL TOTAL,2014-01-01,48930.176837
1,1,GLOBAL TOTAL,2015-01-01,48808.767176
2,2,GLOBAL TOTAL,2016-01-01,49058.441256
3,3,GLOBAL TOTAL,2017-01-01,49878.733809
4,4,GLOBAL TOTAL,2018-01-01,51026.627449


In [16]:
ghg_df['Country'].unique()

array(['GLOBAL TOTAL', 'China', 'United States', 'India', 'EU27',
       'Russia', 'Brazil', 'Indonesia', 'Japan', 'Iran', 'Saudi Arabia',
       'Canada'], dtype=object)

In [14]:
# Load US & Japan manifesto word frequency
manifesto_df = pd.read_csv("../data/processed/manifesto_us_japan_word_freq.csv")
print("Manifesto dataset preview:")
display(manifesto_df.head())

# Load NDC word counts
ndc_df = pd.read_csv("../data/processed/ndc_word_counts.csv")
print("NDC word counts preview:")
display(ndc_df.head())

Manifesto dataset preview:


,country,year,vocab,count
0,Japan,2014,ghg,0
1,Japan,2014,greenhouse,0
2,Japan,2014,net-zero,0
3,Japan,2014,carbon,0
4,Japan,2017,ghg,0


NDC word counts preview:


,ghg,greenhouse,net-zero,carbon,country
0,7,29,0,21,EU2023
1,1,43,0,36,US2021
2,3,58,0,33,US2024


In [18]:
manifesto_df ['country'].unique()

array(['Japan', 'United States'], dtype=object)